The following is the code for scraping LinkedIn for STEM students, find the general skills and degree related.
I would use Beautiful soup
|job title|major|skills|

Notes:
1. may not be a good idea to use requests, because the account might be blocked easily

# Selenium+BeautifulSoup

In [5]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()
from warnings import warn
from datetime import datetime


In [6]:
# replace variables here.
url = "https://www.linkedin.com/jobs/search?keywords=physical%20Chemistry&location=United%20States&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0"
no_of_jobs = 25

In [7]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome('/Users/livi/Git/Web_Scraping/chromedriver')
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [8]:
# to show more jobs. Depends on number of jobs selected
i = 2
while i <= (no_of_jobs/25): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(5)

In [9]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 24 jobs.


In [10]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:

    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

In [11]:
# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)

    # job description    
    jb = driver.find_element_by_class_name('show-more-less-html')
    jb_html=jb.get_attribute('outerHTML')
    del jb
    jb = BeautifulSoup(jb_html, 'html.parser')
    jj = jb.find('div', class_ = 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5')
    full_description='\n'.join(jj.find_all(text=True))
    job_desc.append(full_description)

    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)

    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    industries.append(industry_type)

In [12]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

24
24
24
24
24
24
24
24
24
24


In [13]:
# creating a dataframe
job_data = pd.DataFrame.from_dict({'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Description': job_desc,
'Level': level,
'Type': emp_type,'Function': functions,'Industry': industries},orient='index').T


# cleaning description column
#job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())
job_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job ID        24 non-null     object
 1   Date          24 non-null     object
 2   Company Name  24 non-null     object
 3   Post          24 non-null     object
 4   Location      24 non-null     object
 5   Description   24 non-null     object
 6   Level         24 non-null     object
 7   Type          24 non-null     object
 8   Function      24 non-null     object
 9   Industry      24 non-null     object
dtypes: object(10)
memory usage: 2.0+ KB
None


,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,2398274540,2021-01-28,SGS &amp; Co,Chemistry Analyst I,"Lincolnshire, IL","Job Description\nSUMMARY \nThe Analyst I, Chem...",Associate,Full-time,ResearchAnalystInformation Technology,ConstructionOil & EnergyFinancial Services
1,2406102446,2021-02-03,Treeline Analytics,Laboratory Analyst,"Bellingham, WA","Job Summary\nConducts analysis, experimentatio...",Entry level,Full-time,ResearchAnalystInformation Technology,Environmental Services
2,2406509619,2021-02-03,State of Rhode Island,Environmental Scientist,"Providence, RI",GENERAL STATEMENT OF DUTIES: To perform variou...,Associate,Full-time,Other,Government AdministrationFinancial ServicesHos...
3,2386710524,2021-01-29,The Estée Lauder Companies Inc.,Associate Scientist,"Americas, TX",Refreshing of all raw material specifications ...,Not Applicable,Full-time,ResearchScience,Consumer GoodsCosmetics
4,2360599921,2020-12-28,Bausch Health Companies Inc.,Chemistry Technician IV,"Rochester, NY",Bausch Health Companies Inc. (NYSE/TSX: BHC) i...,Entry level,Full-time,Research,Pharmaceuticals


In [14]:
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
na='job'+'_'.join(dt_string.split(' ')[0].split('/'))+'_'+'_'.join(dt_string.split(' ')[1].split(':'))+'.csv'
job_data.to_csv(na)

# Request+BeautifulSoup

In [4]:
#import python libraries
from bs4 import BeautifulSoup #to parse the page and search for specific elements
import requests #to connect to the web
import re #regular expression
import urllib.request #download images from the urls
import pandas as pd
from pandas import DataFrame
#request with python beautifulsoup using URL,RESPONSE AND SOUPCONTENT
url = "https://www.yellowpages.com/search?search_terms=dentists&geo_location_terms=San+Francisco%2C+CA"
try:
    response = requests.get(url)
    soupcontent = BeautifulSoup(response.content, 'html.parser') #prints out the pagesource
    print(soupcontent)
except:
    raise

">Soderstrom, Susan DDS MS</a></h2><div class="phone">(415) 731-4343</div><div class="categories"><a data-analytics='{"click_id":1171,"adclick":false}' href="/san-francisco-ca/dentists">Dentists</a></div><p class="adr">595 Buckingham Way, San Francisco, CA 94132</p><span class="slogan">Contact Us For Information</span><div class="links"><a data-analytics='{"click_id":13}' href="/listings/1001637354622/directions">Directions</a><a data-analytics='{"click_id":7}' href="/san-francisco-ca/mip/soderstrom-susan-dds-ms-11159746?lid=1001637354622">More Info</a></div></div></div></div></section><div id="banner-ad-bottom-right"><div id="div-gpt-ad-1555631981784-0"><script type="text/javascript">googletag.cmd.push(function() { googletag.display('div-gpt-ad-1555631981784-0'); });

</script></div></div></div></aside><div id="leaderboard"><div id="banner-bottom"><div id="div-gpt-ad-1555630445292-0"><script type="text/javascript">googletag.cmd.push(function() { googletag.display('div-gpt-ad-155563044

In [8]:
#select the container with all the 30 different dentist 
body_dentist = soupcontent.find('div', {'class': 'search-results organic'})
#select individual dentist listings
each_dentist = body_dentist.find_all('div', {'class': 'result'})
print(each_dentist) 
#Name, occupation, reviews, address, and phone numbers)

-bd75-28ca7cd6b6e1","listing_id":"1764937","item_id":-1,"listing_type":"free","ypid":"1764937","content_provider":"MDM","srid":"L-webyp-e3de0f2e-daa9-4d97-ad47-861e89262a70-1764937","item_type":"DOC","lhc":"8008970","ldir":"SF","rate":4.98,"hasTripAdvisor":false,"mip_claimed_status":"mip_unclaimed","mip_ypid":"1764937","click_id":523,"listing_features":"scheduling"}' href="https://patients.doctor.com/directAppointment?provider=538372&amp;partner=YP" rel="nofollow noopener" target="_blank"><svg xmlns="http://www.w3.org/2000/svg"><use xlink:href="#make-an-appointment"></use></svg>Make an Appointment</a></div></div></div></div></div>, <div class="result" data-analytics='{"adclick":true,"events":"event7,event6","category":"8008970","impression_id":"cc120329-3618-4f0e-a082-aa5a0800995d","listing_id":"472748713","item_id":-1,"listing_type":"free","ypid":"472748713","content_provider":"MDM","srid":"L-webyp-e3de0f2e-daa9-4d97-ad47-861e89262a70-472748713","item_type":"listing","lhc":"8008970","

In [14]:
alldentist = []
for i in each_dentist[:20]: #to print only first 20
    name = i.find('a', {'class': 'business-name'}).get_text()
    occupation = i.find('a', {'href': 'https://www.yellowpages.com/san-francisco-ca/dentists'}).get_text()
    reviews = i.find('div', {'class': 'ratings'}).get_text()
    address= i.find('div', {'class': 'street-address'}).get_text()
    address1 = i.find('div', {'class': 'locality'}).get_text()
    address2 = address +", " + address1
    phone_numbers = i.find('div', {'class': 'phones phone primary'}).get_text()
    images = i.find('img')['src']
    dentist = [name, occupation, reviews, address2, phone_numbers, images]


    alldentist.append(dentist)

In [15]:
#Save the data in a Dictionary using DataFrame and then to a csv file, using name column, occupation column, reviews column, address column, and phone numbers column

dentistdf = pd.DataFrame(alldentist,columns=['Name','Occupation','Reviews','Address','Phone Numbers','Images'], index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
print(dentistdf)


#convert to csv
dentistdf.to_csv('dentisttrial.csv', columns=['Name','Occupation','Reviews','Address','Phone Numbers','Images'], index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]) 


                                     Name Occupation            Reviews  \
1                       Schulz John W DDS   Dentists                (1)   
2                       Chin, Gayle A DDS   Dentists                      
3                Jacobson, Theodore E DDS   Dentists                (1)   
4                        Choi Richard DMD   Dentists                      
5               San Francisco Endodontics   Dentists                      
6                      Wong, Ralan DDS MS   Dentists                      
7                  Melitas Euridice A DDS   Dentists                      
8              Gabany, Joseph A R DMD MSD   Dentists                      
9                 Gilberto A Gonzalez DDS   Dentists                      
10                      Wagnild & Mueller   Dentists                      
11                         Mercedes L Tan   Dentists                      
12               Soderstrom, Susan DDS MS   Dentists                      
13                      S